In [130]:
import cobra.test
import os
from os.path import join
from cobra.io import read_sbml_model
model = cobra.io.load_json_model(join("RehMBEL1391_sbml_L3V1.json"))
from cobra import Model, Reaction, Metabolite

# Adding D-glucose to the medium

The model we are using has D-fructorse as carbon source.

## Standard medium content of the model

In [131]:
model.medium   #EX_fru_e is D-frucose

{'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_pi_e': 100.0,
 'EX_cobalt2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_k_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_fru_e': 3.0,
 'EX_nh4_e': 10.0,
 'EX_na_e': 10.0,
 'EX_o2_e': 18.5,
 'EX_mobd_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 100.0}

We will be trying to include D-glucose in the medium and look at the bimass and production of PHB.

With the current medium the biomass and yeald of phb

In [142]:
with model:
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    maximum_yield = phb_production / (-1*(model.reactions.EX_fru_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol fructose') 

The current theoretical maximum biomass productivity: 0.2852858664276781 h
----------------
The current maximum theoretical productivity for making PHB: 3.7627118644067745 mmol/gDW*h
----------------
Maximum theoretical yield = 1.2542372881355914 mmol phb/mmol fructose


### To add D-glucose to the medium we need new reactions to make R.eutropha take up and use D-glucose as a carbon source. 

In [30]:
reaction = Reaction('EX_glc_D_e')
reaction.name = 'D-glucose transport via PEP:Pyr PTS'
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [31]:
EX_glc_D_e = Metabolite(
    'EX_glc_D_e',
    formula='C6H12O6',
    name='D-glucose',
    compartment='c')

In [32]:
reaction.add_metabolites({
    EX_glc_D_e: -1.0,
    model.metabolites.pep_c: -1.0,
    model.metabolites.pyr_c: 1.0,
    model.metabolites.g6p_c: 1.0
})
reaction.reaction  # This gives a string representation of the reaction

'EX_glc_D_e + pep_c --> g6p_c + pyr_c'

In [151]:
with model:
    medium = model.medium
    medium['EX_glc_e'] = 3
    medium['EX_fru_e'] = 0
    print(medium)
    model.medium=medium
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    maximum_yield = phb_production / (-1*(model.reactions.EX_fru_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol Fructose')
    
    medium = model.medium
    medium['EX_glc_e'] = 0
    medium['EX_fru_e'] = 3
    print(medium)
    model.medium=medium

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 3}


KeyError: 'EX_glc_e'

In [145]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    medium['EX_succ_e'] = 10
    print(medium)
    model.medium=medium
    
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    maximum_yield = phb_production / (-1*(model.reactions.EX_succ_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol Succinate')
    
    medium = model.medium
    medium['EX_fru_e'] = 3
    medium['EX_succ_e'] = 0
    print(medium)
    model.medium=medium 

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_succ_e': 10}
The current theoretical maximum biomass productivity: 0.5343325098873305 h
----------------
The current maximum theoretical productivity for making PHB: 6.4545454545454515 mmol/gDW*h
----------------
Maximum theoretical yield = 0.6454545454545452 mmol phb/mmol Succinate
{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_succ_e': 0, 'EX_so4_e': 10.0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_fru_e': 3}


In [108]:
#model.reactions.get_by_id('EX_succ_e')

In [106]:
#for metabolites in model.metabolites.query('succ','name'):
#    print(metabolites.name)

In [107]:
#model.reactions.query('succ','name')

In [148]:
with model:
    medium = model.medium
    medium['EX_o2_e'] = 0
    print(medium)
    model.medium=medium
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    maximum_yield = phb_production / (-1*(model.reactions.EX_fru_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol Fructose')
    
    medium = model.medium
    medium['EX_o2_e'] = 10
    print(medium)
    model.medium=medium 

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 3.0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 0, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0}
3.0
The current theoretical maximum biomass productivity: 0.0 h
----------------
The current maximum theoretical productivity for making PHB: 1.6363636363636365 mmol/gDW*h
----------------
Maximum theoretical yield = 0.5454545454545455 mmol phb/mmol Fructose
{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 3.0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_o2_e': 10}


In [ ]:
###########See if it makes more of phb